### Build a Simple LLM Application with LCEL
Here we'll build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

Here we will be:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [4]:
### Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [6]:

from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000279039DA750>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000279039E0DD0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [8]:
result

AIMessage(content='Here are a few ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (formal)\n* **Salut, comment vas-tu ?** (informal, singular)\n* **Salut, comment allez-vous ?** (informal, plural)\n\n\nThe best option depends on the context and who you\'re speaking to. \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 21, 'total_tokens': 106, 'completion_time': 0.154545455, 'prompt_time': 0.002140117, 'queue_time': 0.021315731, 'total_time': 0.156685572}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a6e472c3-9b93-4ad5-9130-f449a2b5dc62-0', usage_metadata={'input_tokens': 21, 'output_tokens': 85, 'total_tokens': 106})

In [9]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Here are a few ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (formal)\n* **Salut, comment vas-tu ?** (informal, singular)\n* **Salut, comment allez-vous ?** (informal, plural)\n\n\nThe best option depends on the context and who you\'re speaking to. \n'

In [10]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'Bonjour, comment allez-vous ? \n'

In [14]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Translate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [15]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [16]:
result.to_messages()

[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [18]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'